<a href="https://colab.research.google.com/github/matteolimoncini/glasses_or_no_glasses/blob/develop/Glasses_or_not.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing usefull libraries

In [4]:
!pip install dask
!python -m pip install 'fsspec>=0.3.3'
!pip install --user kaggle

     |████████████████████████████████| 118 kB 7.6 MB/s 


Import modules

In [5]:
from dask import dataframe as dd
import copy
import tensorflow as tf

In [6]:
#probably useless
!export KAGGLE_USERNAME=matteolimoncini
!export KAGGLE_KEY=c4df437bb6e9108ad29ae9744b503c89

Setting kaggle API

In [7]:
!mkdir /root/.kaggle
!echo '{"username":"matteolimoncini","key":"c4df437bb6e9108ad29ae9744b503c89"}' > /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

Download and unzip dataset

In [8]:
!kaggle datasets download -d jeffheaton/glasses-or-no-glasses
!unzip glasses-or-no-glasses.zip -d glasses-or-no-glasses

Output streaming troncato alle ultime 5000 righe.
  inflating: glasses-or-no-glasses/faces-spring-2020/faces-spring-2020/face-100.png  
  inflating: glasses-or-no-glasses/faces-spring-2020/faces-spring-2020/face-1000.png  
  inflating: glasses-or-no-glasses/faces-spring-2020/faces-spring-2020/face-1001.png  
  inflating: glasses-or-no-glasses/faces-spring-2020/faces-spring-2020/face-1002.png  
  inflating: glasses-or-no-glasses/faces-spring-2020/faces-spring-2020/face-1003.png  
  inflating: glasses-or-no-glasses/faces-spring-2020/faces-spring-2020/face-1004.png  
  inflating: glasses-or-no-glasses/faces-spring-2020/faces-spring-2020/face-1005.png  
  inflating: glasses-or-no-glasses/faces-spring-2020/faces-spring-2020/face-1006.png  
  inflating: glasses-or-no-glasses/faces-spring-2020/faces-spring-2020/face-1007.png  
  inflating: glasses-or-no-glasses/faces-spring-2020/faces-spring-2020/face-1008.png  
  inflating: glasses-or-no-glasses/faces-spring-2020/faces-spring-2020/face-1009.

Reading training data using dask, that it's usefull because doesn't store in memory all the csv but use batch approach

In [9]:
dask_df = dd.read_csv('./glasses-or-no-glasses/train.csv')
type(dask_df)

dask.dataframe.core.DataFrame

In [10]:
dask_df.head()

id       v1       v2       v3  ...     v510     v511     v512  glasses
0   1  0.37797 -0.94808  0.01346  ...  1.08635  1.69027  0.61609        0
1   2  0.07609 -0.09774  0.39666  ...  0.00866 -1.27640 -0.60496        1
2   3  1.19391 -0.68707 -0.68422  ... -0.60177  0.43555  0.41982        1
3   4  1.34949 -0.31498 -1.30248  ...  0.83997 -0.46986  0.06755        0
4   5 -0.03512 -0.34196  0.14230  ...  0.83641  0.59756 -0.20298        0

[5 rows x 514 columns]

In [27]:
df_glasses = dask_df[dask_df.glasses ==1]
df_no_glasses = dask_df[dask_df.glasses ==0]

len_glasses = len(df_glasses)
len_no_glasses = len(df_no_glasses)
len_dataset = len(dask_df)

print("Number of images: "+ str(len_dataset))
print("Number of images with glasses: "+ str(len_glasses))
print("Number of images without glasses: "+str(len_no_glasses))

Number of images: 4500
Number of images with glasses: 2856
Number of images without glasses: 1644


In [28]:
perc_glasses=round(len_glasses/len_dataset*100,2)
perc_no_glasses=round(len_no_glasses/len_dataset*100,2)
print("Dataset has "+ str(perc_glasses) + "% of images with glasses and " + str(perc_no_glasses) + "% without")

Dataset has 63.47% of images with glasses and 36.53% without


Dataset is not so unbalanced

I create a deep copy of the dataset withoud id and labels, now we have a dataframe that contains only vectors that rapresents image

In [30]:
dataset_only_image =  copy.deepcopy(dask_df)
dataset_only_image.pop('glasses')
dataset_only_image.pop('id')
print(dataset_only_image.head())

        v1       v2       v3       v4  ...     v509     v510     v511     v512
0  0.37797 -0.94808  0.01346  0.17893  ...  0.43507  1.08635  1.69027  0.61609
1  0.07609 -0.09774  0.39666 -0.39026  ... -1.24035  0.00866 -1.27640 -0.60496
2  1.19391 -0.68707 -0.68422 -0.36378  ... -0.58175 -0.60177  0.43555  0.41982
3  1.34949 -0.31498 -1.30248  0.50278  ...  0.01689  0.83997 -0.46986  0.06755
4 -0.03512 -0.34196  0.14230  1.50513  ... -0.29702  0.83641  0.59756 -0.20298

[5 rows x 512 columns]


Print dask_df to assert that has already id and labels

In [29]:
print(dask_df.head())

   id       v1       v2       v3  ...     v510     v511     v512  glasses
0   1  0.37797 -0.94808  0.01346  ...  1.08635  1.69027  0.61609        0
1   2  0.07609 -0.09774  0.39666  ...  0.00866 -1.27640 -0.60496        1
2   3  1.19391 -0.68707 -0.68422  ... -0.60177  0.43555  0.41982        1
3   4  1.34949 -0.31498 -1.30248  ...  0.83997 -0.46986  0.06755        0
4   5 -0.03512 -0.34196  0.14230  ...  0.83641  0.59756 -0.20298        0

[5 rows x 514 columns]


We create a dataframe that contains only labels

In [31]:
labels = dask_df[['glasses']]

In [32]:
labels.head()

glasses
0        0
1        1
2        1
3        0
4        0

In [34]:
dataset = tf.data.Dataset.from_tensor_slices((dataset_only_image.values,labels.values))

In [36]:
model = tf.keras.Sequential(
  [
      tf.keras.layers.Input(shape=(512,)),
      tf.keras.layers.Dense(1, activation='softmax')
  ])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# print model layers
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 513       
Total params: 513
Trainable params: 513
Non-trainable params: 0
_________________________________________________________________


In [37]:
train_dataset = dataset.shuffle(2000).batch(1)

In [38]:
model.fit(train_dataset, epochs=10)

Epoch 1/10
4500/4500 [==============================] - 5s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6347
Epoch 2/10
4500/4500 [==============================] - 5s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6347
Epoch 3/10
4500/4500 [==============================] - 5s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6347
Epoch 4/10
4500/4500 [==============================] - 5s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6347
Epoch 5/10
4500/4500 [==============================] - 5s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6347
Epoch 6/10
4500/4500 [==============================] - 5s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6347
Epoch 7/10
4500/4500 [==============================] - 4s 993us/step - loss: 0.0000e+00 - accuracy: 0.6347
Epoch 8/10
4500/4500 [==============================] - 5s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6347
Epoch 9/10
4500/4500 [==============================] - 5s 1ms/step - loss: 0.0000e+00 - accuracy: 0.6347
Epoch 10/10
4500/4500 [=====================